In [14]:
import requests
import bs4
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import numpy as np
import re
import pandas as pd
import souper_v2
import time

In [7]:
sampled_players = pd.read_csv('/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/Final_datasets/final_list_of_sampled_players.csv')

In [8]:
sampled_players.head()

,player,lastname,team,link_tm,link_ws
0,Max Clark,Clark,Vitesse,/max-clark/profil/spieler/183291,/Players/313996/Show/Max-Clark
1,Sergio Postigo,Postigo,Levante,/sergio-postigo/profil/spieler/158791,/Players/109896/Show/Sergio-Postigo
2,Stanislav Iljutcenko,Iljutcenko,Duisburg,/stanislav-iljutcenko/profil/spieler/191292,/Players/132901/Show/Stanislav-Iljutcenko
3,Iván Marcone,Marcone,Boca Juniors,/ivan-marcone/profil/spieler/90451,/Players/125246/Show/Iván-Marcone
4,Florent Mollet,Mollet,Montpellier,/florent-mollet/profil/spieler/222859,/Players/114169/Show/Florent-Mollet


In [9]:
sampled_players.shape

(1200, 5)

In [10]:
sp = souper_v2.souper()

In [11]:
def get_google_results_webdriver(list_players):

    g_results = {'link':[], 'search':[], 'results':[]}
    pattern = re.compile(r'(?<=About ).+(?= results)')

    for each in tqdm_notebook(range(len(list_players.player))):
        each_team = list_players.team[each].replace(" ","+")
        each_name = list_players.player[each].replace(" ","+")
        each_search = each_name + "+" + each_team
        url = "https://www.google.com/search?q={}".format(each_search)
        soup = sp.get_soup(url)
        results = soup.find('div', attrs={'id':'resultStats'}).text.strip()
        results = re.findall(pattern, results)
    
        try:
            results.replace(",","")
        except:
            pass
        g_results['link'].append(list_players.link_tm[each])
        g_results['search'].append(each_search)
        g_results['results'].append(results[0])
        
    return g_results

In [12]:
def get_google_results(list_players):

    g_results = {'link':[], 'search': [], 'results':[]}
    pattern = re.compile(r'(?<=About ).+(?= results)')

    for each in tqdm_notebook(range(len(list_players.player))):
        each_team = list_players.team[each].replace(" ","+")
        each_name = list_players.player[each].replace(" ","+")
        each_search = each_name + "+" + each_team       
        url = "https://www.google.com/search?q={}".format(each_search)
        r = requests.get(url)
        soup = BeautifulSoup(r.text,'html.parser')
        
        try:
            results = soup.find('div', attrs={'id':'resultStats'}).text.strip()
            results = re.findall(pattern, results)
        except:
            time.sleep(20)
            r = requests.get(url)
            soup = BeautifulSoup(r.text,'html.parser')
            try:
                results = soup.find('div', attrs={'id':'resultStats'}).text.strip()
                results = re.findall(pattern, results)
            except:
                results = np.nan

        try:
            results.replace(",","")
        except:
            pass
        
        g_results['link'].append(list_players.link_tm[each])
        g_results['search'].append(each_search)
        try:
            g_results['results'].append(results[0])
        except:
            g_results['results'].append(results)
        
    return g_results

In [19]:
results_df = get_google_results(sampled_players)

In [36]:
test = pd.DataFrame(results_df)

In [37]:
test['results'] = test.results.map(lambda x: str(x).replace(',',''))

In [38]:
test['results'] = test.results.map(lambda x: str(x).replace('nan','0'))

In [39]:
test['results'] = pd.to_numeric(test['results'])

In [40]:
test.head()

,link,search,results
0,/max-clark/profil/spieler/183291,Max+Clark+Vitesse,1290000
1,/sergio-postigo/profil/spieler/158791,Sergio+Postigo+Levante,173000
2,/stanislav-iljutcenko/profil/spieler/191292,Stanislav+Iljutcenko+Duisburg,67900
3,/ivan-marcone/profil/spieler/90451,Iván+Marcone+Boca+Juniors,183000
4,/florent-mollet/profil/spieler/222859,Florent+Mollet+Montpellier,190000


In [42]:
test.shape

(1200, 3)

In [44]:
test.to_csv(r'/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/Final_datasets/google_results_def_v2.csv',index=False)


In [48]:
test_reader = pd.read_csv('/Users/gonzaloferreiro/Documents/GA_Materials/DS_Inmersive/DSI8-lessons/projects/project-capstone/Moving_forward/Final_datasets/google_results_def_v2.csv')


In [49]:
test_reader.shape

(1200, 3)

In [50]:
test_reader.head()

,link,search,results
0,/max-clark/profil/spieler/183291,Max+Clark+Vitesse,1290000
1,/sergio-postigo/profil/spieler/158791,Sergio+Postigo+Levante,173000
2,/stanislav-iljutcenko/profil/spieler/191292,Stanislav+Iljutcenko+Duisburg,67900
3,/ivan-marcone/profil/spieler/90451,Iván+Marcone+Boca+Juniors,183000
4,/florent-mollet/profil/spieler/222859,Florent+Mollet+Montpellier,190000
